In [1]:
import numpy as np
from scipy.optimize import minimize

In [25]:
def obj(xy):
    # xy[0] is x location
    # xy[1] is y location
    
    distance1 = np.sqrt((5-xy[0])**2 + (10-xy[1])**2)
    distance2 = np.sqrt((10-xy[0])**2 + (5-xy[1])**2)
    distance3 = np.sqrt((0-xy[0])**2 + (12-xy[1])**2)
    distance4 = np.sqrt((12-xy[0])**2 + (0-xy[1])**2)
    
    return 2*(distance1*200 + distance2*150 + distance3*200 + distance4*300) 

In [26]:
print(obj([0,0]))
print(obj([3,3]))

19826.237921249263
14582.9099030015


In [27]:
best_loc = minimize(obj,[0,0],method='SLSQP')

In [28]:
best_loc.x

array([9.31414739, 5.02869991])

In [29]:
best_loc.fun

10913.079375064093

In [7]:
# inequality constraints must be >= 0! 
# that means the constraint that x[1] >= x[0] must be written as x[1] - x[0] >= 0
# then we just need a function to return x[1]-x[0]
# and the solver will know that this should be >= 0
# it works similary for equality constraints...
# equality constraints must be written as a function(x) = 0


def confun(x):
    return x[1]-x[0]

In [8]:
# now we create a dictionary telling the solver what type of constraint and what the function is
constr1 = {'type':'ineq', 'fun': confun}
# then we put all our constraint dictionaries into a list
constraints = [constr1]

In [22]:
best_loc_north = minimize(obj,[3,2],constraints=constraints)

In [23]:
best_loc_north.x

array([6.93579245, 6.93579245])

In [24]:
best_loc_north.fun

11124.981578185989

In [12]:
# if you have an inequality constraint, it's usually best for your initial guess
# to be at a point that does not satisfy the inequality constraint at equality
# for example, if you have a constraint x2 >= x1, it's best not to start at x2 = x1.
#
# but if you have an equality constraint, you should start at a point that satisfies
# that constraint
# for example if you have a constraint x1+x2 = 1, then you should start somewhere 
# like x1 = 0.4, x2 = 0.6


# Jacobian
If we can do a little calculus then we can do better!

The jacobian is the vector of partial derivatives

If we can calculate the jacobian of the objective and each constraint the solver will work better

In [13]:
def con_jac(xy):
    return [-1,1]

In [14]:
con1 = constr1 = {'type':'ineq', 'fun': confun, 'jac': con_jac}
con = [con1]

In [15]:
def obj_jac(xy):
    dd1dx = 0.5/np.sqrt((5-xy[0])**2 + (10-xy[1])**2)*2*(xy[0]-5)
    dd1dy = 0.5/np.sqrt((5-xy[0])**2 + (10-xy[1])**2)*2*(xy[1]-10)
    dd2dx = 0.5/np.sqrt((10-xy[0])**2 + (5-xy[1])**2)*2*(xy[0]-10)
    dd2dy = 0.5/np.sqrt((10-xy[0])**2 + (5-xy[1])**2)*2*(xy[1]-5)
    dd3dx = 0.5/np.sqrt((0-xy[0])**2 + (12-xy[1])**2)*2*(xy[0]-0)
    dd3dy = 0.5/np.sqrt((0-xy[0])**2 + (12-xy[1])**2)*2*(xy[1]-12)
    dd4dx = 0.5/np.sqrt((12-xy[0])**2 + (0-xy[1])**2)*2*(xy[0]-12)
    dd4dy = 0.5/np.sqrt((12-xy[0])**2 + (0-xy[1])**2)*2*(xy[1]-0)
    
    return [2*(dd1dx*200 + dd2dx*150 + dd3dx*200 + dd4dx*300), 2*(dd1dy*200 + dd2dy*150 + dd3dy*200 + dd4dy*300)]

In [16]:
best_loc_north_withJacobian = minimize(obj,[3,2],constraints=con,jac=obj_jac)

In [17]:
best_loc_north_withJacobian.fun

11124.981578187218

In [18]:
best_loc_north_withJacobian.x

array([6.93579262, 6.93579262])